### **Tahap 2 Disusun Oleh** 👨‍🎓✒️
>**Nama: Fitra Romeo Winky**

>**NIM: 202210370311056**

>**Nama: Muhammad Aunul Hakim**

>**NIM: 202210370311073**

>**Kelas: C**

>**Mata Kuliah: Penalaran Komputer**

>**Jurusan: Informatika**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import re

# === PATH SETUP ===
BASE_DIR = '/content/drive/MyDrive/SEMESTER 6/Penalaran Komputer/UAS_Penalaran Komputer'
CSV_INPUT = os.path.join(BASE_DIR, 'CSV/putusan_ma_penganiayaan_2025-06-26.csv')  # ganti sesuai nama
RAW_DIR = os.path.join(BASE_DIR, 'data/raw')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')
os.makedirs(PROCESSED_DIR, exist_ok=True)

CSV_OUTPUT = os.path.join(PROCESSED_DIR, 'cases.csv')
JSON_OUTPUT = os.path.join(PROCESSED_DIR, 'cases.json')

# === BACA METADATA DARI CSV ===
df_meta = pd.read_csv(CSV_INPUT)

# === PROSES TIAP DOKUMEN ===
case_data = []

def extract_ringkasan_fakta(text):
    # Ambil paragraf awal sebagai ringkasan kasar
    return text.strip()[:500]

def extract_argumen_hukum(text):
    # Ambil bagian yang mengandung kata "menimbang", "putusan", "memutuskan"
    match = re.search(r'(menimbang.*?)\n', text.lower(), re.DOTALL)
    return match.group(1)[:500] if match else ''

for idx, row in df_meta.iterrows():
    pdf_name = row.get('file_name_pdf', '')
    if not isinstance(pdf_name, str):
        continue

    txt_filename = pdf_name.replace('.pdf', '.txt').replace("/", "_")
    txt_path = os.path.join(RAW_DIR, txt_filename)

    if os.path.exists(txt_path):
        with open(txt_path, 'r', encoding='utf-8') as f:
            text_full = f.read()

        # Ekstraksi konten kunci
        ringkasan_fakta = extract_ringkasan_fakta(text_full)
        argumen_hukum = extract_argumen_hukum(text_full)

        # Feature Engineering
        length_kata = len(text_full.split())
        jumlah_pasal = text_full.lower().count("pasal")
        contains_pidana = "pidana" in text_full.lower()
        contains_penganiayaan = "penganiayaan" in text_full.lower()

        # Bangun record
        case_data.append({
            'case_id': idx + 1,
            'no_perkara': row.get('nomor', ''),
            'tanggal': row.get('tanggal_register', ''),
            'jenis_perkara': row.get('klasifikasi', ''),
            'pasal': row.get('kaidah', ''),
            'pihak': row.get('lembaga_peradilan', ''),
            'ringkasan_fakta': ringkasan_fakta,
            'argumen_hukum': argumen_hukum,
            'length_kata': length_kata,
            'jumlah_pasal': jumlah_pasal,
            'mengandung_pidana': contains_pidana,
            'mengandung_penganiayaan': contains_penganiayaan,
            'text_full': text_full.strip()
        })

# === SIMPAN KE CSV & JSON ===
df_output = pd.DataFrame(case_data)
df_output.to_csv(CSV_OUTPUT, index=False, encoding='utf-8')
df_output.to_json(JSON_OUTPUT, orient='records', indent=2, force_ascii=False)

print(f"✔️ Tahap 2 selesai dan disimpan:")
print(f"- CSV: {CSV_OUTPUT}")
print(f"- JSON: {JSON_OUTPUT}")


✔️ Tahap 2 selesai dan disimpan:
- CSV: /content/drive/MyDrive/SEMESTER 6/Penalaran Komputer/UAS_Penalaran Komputer/data/processed/cases.csv
- JSON: /content/drive/MyDrive/SEMESTER 6/Penalaran Komputer/UAS_Penalaran Komputer/data/processed/cases.json


In [ ]:
print(os.listdir(PROCESSED_DIR))


['cases.csv', 'cases.json']
